In [1]:
# Import Library Files

import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torchvision.models import resnet50
from PIL import Image
from tqdm import tqdm

In [2]:
#----
#1. DEFT Module Components
#----

class LocalizationNetwork(nn.Module):
    def __init__(self, input_channels):
        super(LocalizationNetwork, self).__init__()
        self.conv1 = nn.Conv2d(input_channels, 8, kernel_size=7)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(8, 10, kernel_size=5)

        # Placeholder fc1 (will update after shape calculation)
        self.fc1 = nn.Linear(1, 32)  # Temporary placeholder
        self.fc2 = nn.Linear(32, 6)  # 6 affine parameters

        # Initialize weights for identity transformation
        self.fc2.weight.data.zero_()
        self.fc2.bias.data.copy_(torch.tensor([1, 0, 0, 0, 1, 0], dtype=torch.float))

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))

        # Dynamically compute the flattened size
        if not hasattr(self, 'computed_fc1'):
            flattened_size = x.view(x.shape[0], -1).shape[1]
            self.fc1 = nn.Linear(flattened_size, 32).to(x.device)
            self.computed_fc1 = True  # Prevent re-initialization

        x = x.view(x.shape[0], -1)  # Flatten
        x = F.relu(self.fc1(x))
        theta = self.fc2(x)
        theta = theta.view(-1, 2, 3)
        return theta


class WeightingModule(nn.Module):
    def __init__(self, sigma=0.5):
        super(WeightingModule, self).__init__()
        self.lambda_param = nn.Parameter(torch.tensor(0.5))
        self.sigma = sigma

    def forward(self, grid):
        dist2 = grid[..., 0] ** 2 + grid[..., 1] ** 2
        weight = 1 + self.lambda_param * torch.exp(-dist2 / (2 * self.sigma ** 2))
        return weight.unsqueeze(-1)


class DEFTModule(nn.Module):
    def __init__(self, input_channels, sigma=0.5):
        super(DEFTModule, self).__init__()
        self.localization = LocalizationNetwork(input_channels)
        self.weighting = WeightingModule(sigma)

    def forward(self, x):
        theta = self.localization(x)
        grid = F.affine_grid(theta, x.size(), align_corners=False)
        weight = self.weighting(grid)
        x_transformed = F.grid_sample(x, grid, align_corners=False)

        if x.shape[1] > 1:
            weight = weight.expand(-1, x.shape[2], x.shape[3], x.shape[1]).permute(0, 3, 1, 2)
        else:
            weight = weight.permute(0, 3, 1, 2)

        x_weighted = x_transformed * weight
        return x_weighted



In [3]:
#----
#2. ResNet50 Model for Feature Extraction
#----

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

resnet_model = resnet50(weights=True)
resnet_model = torch.nn.Sequential(*list(resnet_model.children())[:-1])  # Remove classification layer
resnet_model.eval().to(device)

# Load DEFT Module
deft_model = DEFTModule(input_channels=3).to(device)  # Input channels = 3 (RGB)
deft_model.eval()

###########################################################
# Define Image Transformation
###########################################################

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


C:\Users\PAWANESH\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [4]:
#----
#3. Feature Extraction Function with DEFT
#----

def extract_features(image_path, transform, device=device):
    """Extract ResNet50 features from a single image after DEFT transformation."""
    try:
        image = Image.open(image_path).convert('RGB')
        image = transform(image).unsqueeze(0).to(device)  # Convert to tensor and send to GPU

        # Apply DEFT Transformation
        with torch.no_grad():
            image = deft_model(image)  # Pass through DEFT Module
            features = resnet_model(image).squeeze().cpu().numpy()  # Extract ResNet50 features
        
        return features
    except Exception as e:
        print(f"Skipping frame: {image_path} due to error: {e}")
        return None


###########################################################
# Paths to RGB and Optical Flow Directories
###########################################################

rgb_path = "D:/Datasets/Datasets/MECCANO/RGB_Frames_Original/0005"
#flow_u_path = "D:/Datasets/Datasets/EPIC_Kitchen/OpticalFlow/P01_02/u"
#flow_v_path = "D:/Datasets/Datasets/EPIC_Kitchen/OpticalFlow/P01_02/v"

label_csv_path = "D:/Datasets/Datasets/MECCANO/Labels_ExcelFile(1-6)/0005_CSV.csv"
labels_df = pd.read_csv(label_csv_path)

output_csv = "../SavedFeatures/Meccano_0005.csv"

In [8]:
###########################################################
# Extract Features with DEFT & Save (RGB only)
###########################################################
S = 10  # Sampling every 2nd frame
features_list = []

all_frames = sorted(os.listdir(rgb_path))[::S]

# ✅ Print matching ranges for verification
print("Checking frame range in CSV...")
for _, row in labels_df.iterrows():
    print(f"StartFrame: {row['StartFrame']}, EndFrame: {row['EndFrame']}, ActionLabel: {row['ActionLabel']}")

for frame in tqdm(all_frames, desc="Extracting Features with DEFT (RGB only)"):
    rgb_frame_path = os.path.join(rgb_path, frame)

    # ✅ Extract frame number correctly from frame name (no +1)
    try:
        frame_number = int(frame.split('_')[-1].split('.')[0])  # No offset
    except ValueError:
        print(f"Skipping invalid frame name: {frame}")
        continue

    # ✅ Print the current frame number for verification
    print(f"Processing frame: {frame} | Frame number: {frame_number}")

    # Extract features from RGB only
    rgb_features = extract_features(rgb_frame_path, transform, device)

    if rgb_features is not None:
        label_row = labels_df[(labels_df['StartFrame'] <= frame_number) & (labels_df['EndFrame'] >= frame_number)]

        if label_row.empty:
            print(f"❌ No match for frame {frame_number}")
            print("Sample of available label ranges:")
            print(labels_df[['StartFrame', 'EndFrame']].head())
            continue
        else:
            action_label = label_row.iloc[0]['ActionLabel']
            print(f"✅ Frame {frame_number} matched with ActionLabel: {action_label}")

        # Add extracted features and action label to the list
        features_list.append([frame, action_label] + rgb_features.tolist())

###########################################################
# Save Extracted Features to CSV
###########################################################
if len(features_list) == 0:
    raise ValueError("No valid features extracted. Please check the dataset paths and feature extraction function.")

# Define column names for the output CSV
columns = ["Frame", "ActionLabel"] + [f"Feature_{i}" for i in range(len(rgb_features))]
df = pd.DataFrame(features_list, columns=columns)

# Save the extracted features to a CSV
df.to_csv(output_csv, index=False)

print(f"✅ Feature extraction completed! Saved to {output_csv}")


Checking frame range in CSV...
StartFrame: 78, EndFrame: 92, ActionLabel: 0
StartFrame: 968, EndFrame: 976, ActionLabel: 29
StartFrame: 993, EndFrame: 1001, ActionLabel: 14
StartFrame: 1006, EndFrame: 1030, ActionLabel: 53
StartFrame: 1109, EndFrame: 1117, ActionLabel: 5
StartFrame: 1111, EndFrame: 1120, ActionLabel: 26
StartFrame: 1139, EndFrame: 1149, ActionLabel: 18
StartFrame: 1150, EndFrame: 1158, ActionLabel: 39
StartFrame: 1154, EndFrame: 1163, ActionLabel: 18
StartFrame: 1176, EndFrame: 1189, ActionLabel: 18
StartFrame: 1183, EndFrame: 1190, ActionLabel: 39
StartFrame: 552, EndFrame: 563, ActionLabel: 0
StartFrame: 1184, EndFrame: 1193, ActionLabel: 5
StartFrame: 1201, EndFrame: 1213, ActionLabel: 13
StartFrame: 1220, EndFrame: 1225, ActionLabel: 51
StartFrame: 1225, EndFrame: 1234, ActionLabel: 18
StartFrame: 1239, EndFrame: 1272, ActionLabel: 52
StartFrame: 1273, EndFrame: 1283, ActionLabel: 18
StartFrame: 1295, EndFrame: 1338, ActionLabel: 52
StartFrame: 1400, EndFrame: 1407

Extracting Features with DEFT (RGB only):   0%|                                        | 2/676 [00:00<00:43, 15.55it/s]

Processing frame: frame_0000.jpg | Frame number: 0
❌ No match for frame 0
Sample of available label ranges:
   StartFrame  EndFrame
0          78        92
1         968       976
2         993      1001
3        1006      1030
4        1109      1117
Processing frame: frame_0010.jpg | Frame number: 10
❌ No match for frame 10
Sample of available label ranges:
   StartFrame  EndFrame
0          78        92
1         968       976
2         993      1001
3        1006      1030
4        1109      1117
Processing frame: frame_0020.jpg | Frame number: 20


Extracting Features with DEFT (RGB only):   1%|▎                                       | 6/676 [00:00<00:43, 15.54it/s]

❌ No match for frame 20
Sample of available label ranges:
   StartFrame  EndFrame
0          78        92
1         968       976
2         993      1001
3        1006      1030
4        1109      1117
Processing frame: frame_0030.jpg | Frame number: 30
❌ No match for frame 30
Sample of available label ranges:
   StartFrame  EndFrame
0          78        92
1         968       976
2         993      1001
3        1006      1030
4        1109      1117
Processing frame: frame_0040.jpg | Frame number: 40
❌ No match for frame 40
Sample of available label ranges:
   StartFrame  EndFrame
0          78        92
1         968       976
2         993      1001
3        1006      1030
4        1109      1117
Processing frame: frame_0050.jpg | Frame number: 50
❌ No match for frame 50
Sample of available label ranges:
   StartFrame  EndFrame
0          78        92
1         968       976
2         993      1001
3        1006      1030
4        1109      1117


...

❌ No match for frame 6650
Sample of available label ranges:
   StartFrame  EndFrame
0          78        92
1         968       976
2         993      1001
3        1006      1030
4        1109      1117
Processing frame: frame_6660.jpg | Frame number: 6660
✅ Frame 6660 matched with ActionLabel: 29
Processing frame: frame_6670.jpg | Frame number: 6670
❌ No match for frame 6670
Sample of available label ranges:
   StartFrame  EndFrame
0          78        92
1         968       976
2         993      1001
3        1006      1030
4        1109      1117
Processing frame: frame_6680.jpg | Frame number: 6680
❌ No match for frame 6680
Sample of available label ranges:
   StartFrame  EndFrame
0          78        92
1         968       976
2         993      1001
3        1006      1030
4        1109      1117
Processing frame: frame_6690.jpg | Frame number: 6690
❌ No match for frame 6690
Sample of available label ranges:
   StartFrame  EndFrame
0          78        92
1         968       97

Extracting Features with DEFT (RGB only): 100%|█████████████████████████████████████▉| 674/676 [00:28<00:00, 23.94it/s]

Processing frame: frame_6710.jpg | Frame number: 6710
❌ No match for frame 6710
Sample of available label ranges:
   StartFrame  EndFrame
0          78        92
1         968       976
2         993      1001
3        1006      1030
4        1109      1117
Processing frame: frame_6720.jpg | Frame number: 6720
✅ Frame 6720 matched with ActionLabel: 26
Processing frame: frame_6730.jpg | Frame number: 6730
❌ No match for frame 6730
Sample of available label ranges:
   StartFrame  EndFrame
0          78        92
1         968       976
2         993      1001
3        1006      1030
4        1109      1117
Processing frame: frame_6740.jpg | Frame number: 6740
❌ No match for frame 6740
Sample of available label ranges:
   StartFrame  EndFrame
0          78        92
1         968       976
2         993      1001
3        1006      1030
4        1109      1117
Processing frame: frame_6750.jpg | Frame number: 6750


Extracting Features with DEFT (RGB only): 100%|██████████████████████████████████████| 676/676 [00:28<00:00, 23.59it/s]


❌ No match for frame 6750
Sample of available label ranges:
   StartFrame  EndFrame
0          78        92
1         968       976
2         993      1001
3        1006      1030
4        1109      1117
✅ Feature extraction completed! Saved to ../SavedFeatures/Meccano_0005.csv


In [9]:
###########################################################
# Display Extracted Features for Verification
###########################################################
f = pd.read_csv("../SavedFeatures/Meccano_0005.csv")
f

,Frame,ActionLabel,Feature_0,Feature_1,Feature_2,Feature_3,Feature_4,Feature_5,Feature_6,Feature_7,...,Feature_2038,Feature_2039,Feature_2040,Feature_2041,Feature_2042,Feature_2043,Feature_2044,Feature_2045,Feature_2046,Feature_2047
0,frame_0080.jpg,0,0.264659,0.660212,1.074581,0.237434,1.745461,0.100952,0.277871,0.723107,...,0.414991,0.277170,0.095533,0.759139,0.420223,0.289117,0.075979,0.105973,0.011653,0.488003
1,frame_0090.jpg,0,0.350643,0.264853,0.386139,0.171208,2.759244,0.196105,0.494262,1.221428,...,0.340227,0.268832,0.070351,0.194222,0.173281,0.187330,0.180391,0.424948,0.060755,0.730003
2,frame_0110.jpg,15,1.000748,0.833303,0.828925,0.356233,0.328324,0.578510,0.346710,0.494077,...,0.315165,0.118660,0.080917,0.885923,0.177255,0.692081,0.187779,0.298714,0.319751,0.201922
3,frame_0120.jpg,0,0.513097,0.635973,0.448294,0.196552,0.882536,0.224773,0.443164,0.753894,...,1.115593,0.500120,0.182359,0.723184,0.343887,0.433635,0.020229,0.211381,0.022715,0.462316
4,frame_0140.jpg,0,0.107099,0.367559,0.129395,0.344745,0.162751,0.255897,0.152504,0.328013,...,0.315199,0.445676,0.080200,0.156318,0.353378,0.271468,0.463918,0.045682,0.503576,0.459803
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
424,frame_6550.jpg,58,0.542285,0.643273,0.222710,0.817138,0.187048,0.660838,0.110984,0.290174,...,0.707568,0.015027,0.014920,0.336055,0.239834,0.214410,0.015061,0.375617,0.232405,0.532271
425,frame_6560.jpg,58,0.455280,0.417055,0.352933,0.651549,0.048613,0.496639,0.087838,0.512411,...,0.798252,0.078754,0.076369,0.200215,0.555751,0.113995,0.121060,0.252357,0.170831,0.373921
426,frame_6570.jpg,58,0.378471,1.353936,0.628499,0.486005,0.100848,0.210818,0.418445,0.582185,...,0.431928,0.043798,0.086475,0.603486,0.407115,0.266935,0.122080,0.828828,0.571565,0.482938
427,frame_6660.jpg,29,0.723809,0.582377,0.323763,0.644180,0.401401,0.352420,0.632854,0.692289,...,0.278361,0.096367,0.102678,0.764079,0.065295,0.174795,0.431573,0.709530,0.333146,0.029281
